# PySpark - Tarea 1

### Practice 1 Task 1: Introduction to RDDs – Transformations and Actions

In [1]:
#pip install pyspark

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext(master = 'local[2]')

23/12/02 19:58:53 WARN Utils: Your hostname, onur-ideacenter resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlo1)
23/12/02 19:58:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/02 19:58:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Question 1
1. Calculate the mean and standard deviation of a file containing heights (alturas_v0.csv)

In [3]:
# Import Data by making all values float and filtering negative values
data = sc.textFile("alturas_v0.csv")
new_data = data.map(lambda x: float(x))
filtered_data = new_data.filter(lambda x: x>=0)

# Convert to cm
my_data = filtered_data.map(lambda x: x * 100 if x < 2 else x if x >= 100 else x * 100)
my_data.take(10)

[173.0, 161.0, 176.0, 180.0, 162.0, 176.0, 166.0, 174.0, 158.0, 162.0]

In [4]:
# Calculate Mean and Std of Data
count_data = my_data.count()
sum_data = my_data.reduce(lambda x, y: x + y)
mean_data = sum_data / count_data
std_data = (my_data.map(lambda x: pow(x - mean_data, 2)).reduce(lambda x, y: x + y) / count_data) ** 0.5

print("Data Mean: ", mean_data)
print("Data Std:  ", std_data)

Data Mean:  168.93825222396651
Data Std:   7.834055783371033


In [5]:
# Calculate Mean and Std of Data using Built-in Functions in Pyspark to Compare
avg_my_data = my_data.mean()
print("Data Mean: ", avg_my_data)
std_my_data = my_data.stdev()
print("Data Std:  ",std_my_data)

Data Mean:  168.93825222396651
Data Std:   7.834055783371039


## Question 2
2. Calculate the mean and standard deviation of a file containing heights (alturas.csv) by gender

In [6]:
# Import Data by making all values float and filtering negative values
data2 = sc.textFile("alturas.csv")
data3 = data2.map(lambda x: x.split(",")).map(lambda x: (x[0], float(x[1])))
data3 = data3.filter(lambda x: (x[1]>=0))

# Convert to cm
data3 = data3.map(lambda x: (x[0], x[1] * 100 if x[1] < 2 else x[1] if x[1] >= 100 else x[1] * 100))
data3.take(10)

[('H', 173.0),
 ('M', 161.0),
 ('H', 176.0),
 ('H', 180.0),
 ('M', 162.0),
 ('H', 176.0),
 ('M', 166.0),
 ('H', 174.0),
 ('M', 158.0),
 ('M', 162.0)]

In [7]:
# Calculate Sum and Count of H and M
data4 = data3.reduceByKey(lambda v1, v2: v1+v2).collect()
count_data4 = sorted(data3.countByKey().items())

# Get the sum and count
sum_h, sum_m = data4[0][1], data4[1][1]
count_h, count_m = count_data4[0][1], count_data4[1][1]

# Calculate the mean
mean_h = sum_h / count_h
mean_m = sum_m / count_m

# Calculate the standard deviation
std_h = (data3.filter(lambda x: x[0] == "H").map(lambda x: pow(x[1] - mean_h, 2)).reduce(lambda x, y: x + y) / count_h) ** 0.5
std_m = (data3.filter(lambda x: x[0] == "M").map(lambda x: pow(x[1] - mean_m, 2)).reduce(lambda x, y: x + y) / count_m) ** 0.5

print("Mean of H: ", mean_h)
print("Mean of M: ", mean_m)
print("Std of H:  ", std_h)
print("Std of M:  ", std_m)

Mean of H:  173.46075433231397
Mean of M:  164.16774193548386
Std of H:   7.5312288366375055
Std of M:   4.684891325217869
